# AWS-RoseTTAFold: Bulk Job Analysis

## I. Introduction

This notebook demonstrates how to analyze multiple protein simultaneously, in this case a subset of the CASP14 target set.

## II. Environment setup

In [ ]:
## Install dependencies
!pip install -q -q -r requirements.txt

## Import helper functions at rfutils/rfutils.py
from rfutils import rfutils

## Load additional dependencies
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import boto3
import glob
import json
from IPython.display import display
import pandas as pd
import sagemaker

pd.set_option("max_colwidth", None)

# Get service clients
session = boto3.session.Session()
sm_session = sagemaker.session.Session()
region = session.region_name
role = sagemaker.get_execution_role()
s3 = boto3.client("s3", region_name=region)
account_id = boto3.client("sts").get_caller_identity().get("Account")

bucket = sm_session.default_bucket()

## III. Input Protein Sequence

Download and process CASP14 sequences

In [ ]:
!wget "https://predictioncenter.org/download_area/CASP14/sequences/casp14.seq.txt" -O "data/casp14.fa"
!sed '137,138d' "data/casp14.fa" > "data/casp14_dedup.fa" # Remove duplicate entry for T1085

casp14_iterator = SeqIO.parse("data/casp14_dedup.fa", "fasta")
casp14_df = pd.DataFrame(
    (
        (record.id, record.description, len(record), record.seq)
        for record in casp14_iterator
    ),
    columns=["id", "description", "length", "seq"],
).sort_values(by="length")
!rm data/casp14*

Display information about CASP14 proteins

In [ ]:
with pd.option_context("display.max_rows", None):
    display(casp14_df.loc[:, ("id", "description")])

Plot distribution of the protein lengths

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
plt.hist(casp14_df.length, bins=50)
plt.ylabel("Sample count")
plt.xlabel("Residue count")
plt.title("CASP-14 Protein Length Distribution")
plt.show()

Get the names of the AWS Batch resources deployed in your account.

In [ ]:
batch_resources = rfutils.get_rosettafold_batch_resources()
batch_resources

Submit analysis jobs for a subset of CASP14 proteins

In [ ]:
protein_count = 3

for row in casp14_df[:protein_count].itertuples(index=False):
    record = SeqRecord(row.seq, id=row.id, description=row.description)
    print(f"Protein sequence for analysis is \n{record.description}")
    job_name = rfutils.create_job_name(record.id)
    print(f"Automatically-generated job name is: {job_name}")
    input_uri = rfutils.upload_fasta_to_s3(record, bucket, job_name)
    two_step_response = rfutils.submit_2_step_job(
        bucket=bucket,
        job_name=job_name,
        data_prep_input_file="input.fa",
        data_prep_job_definition=batch_resources["dataPrepJobDefinition"][0],
        data_prep_queue=batch_resources["dataPrepJobQueue"][0],
        data_prep_cpu=16,
        data_prep_mem=60,
        predict_job_definition=batch_resources["predictJobDefinition"][0],
        predict_queue=batch_resources["predictJobQueue"][0],
        predict_cpu=24,
        predict_mem=90,
        predict_gpu=1,
    )

## IV. Check Status of Data Prep and Prediction Jobs

In [ ]:
rfutils.get_rf_job_info(
    cpu_queue=batch_resources["dataPrepJobQueue"][0],
    gpu_queue=batch_resources["predictJobQueue"][0],
    hrs_in_past=1,
)